# Bias in word embeddings
Word embeddings is a technique in NLP and text mining to represent words in order to be able to compare words based on similarity. The word embeddings are trained on large data sets with text written by humans. Therefore, the bias we have and include in our writungs will be transferred to the word embeddings. This project looks at what bias there are in the swedish word embeddings "Swectors" based on text from Göteborgsposten. They will also be compared with another set of word ebedding that will be trained on antoher set of data.

In [2]:
import bz2
import pandas as pd
import numpy as np
import time

## Importing datasets
Importing the large set of vectors might take a little while, please be patient.

In [2]:
colnames = ["word"] + ["dim" + str(x) for x in range(1,301)]

with bz2.open("swectors.txt.bz2") as source:
    swectors = pd.read_csv(source, header=None, names=colnames, delimiter=" ", skiprows=[0])
    
familjevectors = pd.read_csv("familjevectors.zip", header=None, names=colnames, delimiter=" ", skiprows=[0])

flashvectors = pd.read_csv("flashvectors.zip", header=None, names=colnames, delimiter=" ", skiprows=[0])
    
#swectors.tail()

### Import small sized dataset

colnames = ["word"] + ["dim" + str(x) for x in range(1,301)]
with bz2.open("swectors_short-300dim.txt.bz2") as source:
    swectors_short = pd.read_csv(source, header=None, names=colnames, delimiter=" ", skiprows=[0])
    
swectors_short.tail()

Extract the vector for the word 'kvinna', in order to look att similar words for bias measures.
Get the 300 dimensions from the dataframe, convert to numpy and get the following format: `[[dim1 dim2 ... dim299 dim300]]`, take the first element to get a single list. Save it as a tuple with word first and vector second.

In [3]:
def get_word_vector(vectors, word):
    return (word, vectors.loc[vectors['word'] == word].loc[:, 'dim1':'dim300'].to_numpy()[0])

In [4]:
kvinna_s = get_word_vector(swectors, 'kvinna')
man_s = get_word_vector(swectors, 'man')

kvinna_fam = get_word_vector(familjevectors, 'kvinna')
man_fam = get_word_vector(familjevectors, 'man')

kvinna_flash = get_word_vector(flashvectors, 'kvinna')
man_flash = get_word_vector(flashvectors, 'man')

In [6]:
def cosine_similarity(word1, word2):
    # Takes two vectors and calculates the cosine similarity between them
    # @ is dot product
    v1 = word1[1]
    v2 = word2[1]
    return (v1 @ v2) / (np.linalg.norm(v1)*np.linalg.norm(v2))

In [7]:
print('Swectors:', cosine_similarity(kvinna_s, man_s))
print('Familje:', cosine_similarity(kvinna_fam, man_fam))
print('Flash:', cosine_similarity(kvinna_flash, man_flash))

Swectors: 0.3581906572935491
Familje: 0.1536098160481569
Flash: 0.24343163718296817


## Applying function to whole dataframe
This way of doing the calculation will apply a function to each row of the dataframe, and return a Series of same length with all results in it.

In [8]:
def n_most_similar(vectors, n, word, vectors_filtered=None):
    
    if vectors_filtered is not None:
        df = vectors_filtered
        print("filtered")
    else:
        df = vectors
        print("not filtered")
        
    word_vec = get_word_vector(vectors, word)
    
    def similarity(row):
        row_vec = (row['word'], row.loc['dim1':'dim300'].to_numpy())
        return cosine_similarity(word_vec, row_vec)

    start = time.time()
    similarities = df.apply(similarity, axis=1)

    # Concatenate the top n words (plus the word itself) to the similarity values of each word.
    # Also set the correct coulmn name.
    s1 = df.loc[similarities.nlargest(n+1).index, 'word']
    s2 = similarities.nlargest(n+1)
    similars = pd.concat([s1, s2], axis=1).rename(columns={0: "similarity"})
    end = time.time()
    print("Time elapsed: ", end - start)
    return similars

In [9]:
def gender_similarity_difference(vectors, single_word, plural_word=None):
    
    word_vec_single = get_word_vector(vectors, single_word)
    
    if plural_word is not None:
        word_vec_plural = get_word_vector(vectors, plural_word)
    else:
        word_vec_plural = word_vec_single
    
    res = {}
    res['kvinna'] = cosine_similarity(word_vec_single, get_word_vector(vectors, 'kvinna'))
    res['kvinnan'] = cosine_similarity(word_vec_single, get_word_vector(vectors, 'kvinnan'))
    res['kvinnlig'] = cosine_similarity(word_vec_single, get_word_vector(vectors, 'kvinnlig'))
    
    res['man'] = cosine_similarity(word_vec_single, get_word_vector(vectors, 'man'))
    res['mannen'] = cosine_similarity(word_vec_single, get_word_vector(vectors, 'mannen'))
    res['manlig'] = cosine_similarity(word_vec_single, get_word_vector(vectors, 'manlig'))
    
    res['kvinnor'] = cosine_similarity(word_vec_plural, get_word_vector(vectors, 'kvinnor'))
    res['kvinnorna'] = cosine_similarity(word_vec_plural, get_word_vector(vectors, 'kvinnorna'))
    res['kvinnliga'] = cosine_similarity(word_vec_plural, get_word_vector(vectors, 'kvinnliga'))
    
    res['män'] = cosine_similarity(word_vec_plural, get_word_vector(vectors, 'män'))
    res['männen'] = cosine_similarity(word_vec_plural, get_word_vector(vectors, 'männen'))
    res['manliga'] = cosine_similarity(word_vec_plural, get_word_vector(vectors, 'manliga'))
    
    
    
    return res

In [14]:
gender_similarity_difference(flashvectors, 'sjuksköterska')

{'kvinna': 0.46760799254621666,
 'kvinnan': 0.2755152308911619,
 'kvinnlig': 0.4027155559287831,
 'man': 0.10020534787239237,
 'mannen': 0.1957895923357652,
 'manlig': 0.30929783542786066,
 'kvinnor': 0.14743792842666,
 'kvinnorna': 0.11731806225236029,
 'kvinnliga': 0.17153224173838297,
 'män': 0.12394941960243178,
 'männen': 0.11123897598169297,
 'manliga': 0.10089858305380585}

In [37]:
similars_kvinna_s = n_most_similar(swectors, 10, 'kvinna')
print(similars_kvinna_s)

not filtered
Time elapsed:  29.304189920425415
               word  similarity
542          kvinna    1.000000
2269         flicka    0.778604
646         kvinnan    0.687048
2957          pojke    0.676086
2972           tjej    0.673264
497          person    0.671946
58745  tonårsflicka    0.645879
16134       yngling    0.606193
10872       väninna    0.603767
3471            dam    0.596047
11426      polisman    0.593694


In [38]:
similars_kvinna_fam = n_most_similar(familjevectors, 10, 'kvinna')
print(similars_kvinna_fam)

not filtered
Time elapsed:  14.932774066925049
              word  similarity
1153        kvinna    1.000000
636           tjej    0.689873
1194      människa    0.651235
579         person    0.630777
4849           dam    0.628756
5632          brud    0.600699
673          kille    0.595686
1987        flicka    0.558922
1960       kvinnan    0.552127
8354        snubbe    0.538014
91505  kvinnokropp    0.525616


In [39]:
similars_kvinna_flash = n_most_similar(flashvectors, 10, 'kvinna')
print(similars_kvinna_flash)

not filtered
Time elapsed:  28.88240385055542
         word  similarity
381    kvinna    1.000000
3765      dam    0.768512
181      tjej    0.752394
3568     karl    0.746014
2183   flicka    0.736183
256     kille    0.697204
7938     hona    0.692251
10855     tös    0.690076
9738    donna    0.684831
280    person    0.670705
2510    pojke    0.663436


In [40]:
similars_män_s = n_most_similar(swectors, 10, 'män')
print(similars_män_s)

not filtered
Time elapsed:  31.024073600769043
                word  similarity
417              män    1.000000
322          kvinnor    0.804870
1177          männen    0.771062
1867         flickor    0.727492
2288          pojkar    0.692823
1626       kvinnorna    0.690541
1793          killar    0.656109
186         personer    0.655852
33699  tonårsflickor    0.654169
6182      tonåringar    0.639736
1477          tjejer    0.609658


In [41]:
similars_män_fam = n_most_similar(familjevectors, 10, 'män')
print(similars_män_fam)

not filtered
Time elapsed:  15.799741268157959
           word  similarity
1325        män    1.000000
1211    kvinnor    0.903146
5369     karlar    0.751930
5456     männen    0.721142
1615     killar    0.718297
6734  kvinnorna    0.683599
2570     mammor    0.664039
957      tjejer    0.645968
5301     pappor    0.638359
487   människor    0.629805
2897   svenskar    0.584532


In [42]:
similars_män_flash = n_most_similar(flashvectors, 10, 'män')
print(similars_män_flash)

not filtered
Time elapsed:  29.270676851272583
            word  similarity
257          män    1.000000
217      kvinnor    0.917956
1760      männen    0.794857
263       killar    0.760381
1649   kvinnorna    0.739334
7537      karlar    0.714033
2552     flickor    0.698970
160       tjejer    0.689911
187    människor    0.672047
3079      pojkar    0.670901
3618  feminister    0.660804


## Filtering out all adjectives in the word embeddings
To see bias in adjectives, the dataframe with the swectors is filtered to only keep words that are in the dataframe with adjectves from Språkrådet.

In [24]:
with bz2.open("data/adjektiv.txt.bz2") as source:
    adjectives = pd.read_csv(source)
    
swectors_filtered = swectors.loc[swectors['word'].isin(adjectives['Word'])]

familje_filtered = familjevectors.loc[familjevectors['word'].isin(adjectives['Word'])]

flash_filtered = flashvectors.loc[flashvectors['word'].isin(adjectives['Word'])]

In [26]:
print("swectors:", swectors_filtered.shape)
print("familjeliv:", familje_filtered.shape)
print("flashback:", flash_filtered.shape)

swectors: (14706, 301)
familjeliv: (8591, 301)
flashback: (15820, 301)


In [30]:
similars_kvinna_adj_s = n_most_similar(swectors, 10, 'kvinnan', swectors_filtered)
print(similars_kvinna_adj_s)

filtered
Time elapsed:  2.275986433029175
                  word  similarity
24762     prostituerad    0.474156
11113          anhörig    0.405700
61483   mordmisstänkte    0.405525
15205             döde    0.404474
4035          sexuellt    0.392038
12493      minderåriga    0.381626
90384         anhörige    0.373552
3450          sexuella    0.365668
28865         skyldige    0.361844
79494      mordåtalade    0.357744
154091         demente    0.349829


In [32]:
similars_kvinna_adj_fam = n_most_similar(familjevectors, 10, 'kvinnan', familje_filtered)
print(similars_kvinna_adj_fam)

filtered
Time elapsed:  1.379850149154663
               word  similarity
3115          andre    0.423743
32237      osynlige    0.391812
88839       stilige    0.353628
16825           nye    0.348560
40508        blinde    0.341302
28981          döde    0.325323
22942  prostituerad    0.318310
6166      kvinnliga    0.316340
16734        förste    0.313463
73205   anglosaxisk    0.311411
7659         blonda    0.310431


In [33]:
similars_kvinna_adj_flash = n_most_similar(flashvectors, 10, 'kvinnan', flash_filtered)
print(similars_kvinna_adj_flash)

filtered
Time elapsed:  2.428054094314575
                 word  similarity
2611            andre    0.438010
108294          nakne    0.388559
24685     patriarkala    0.368643
86966       dominante    0.351911
7655    västerländska    0.332195
20863        skyldige    0.329829
155707      välklädde    0.325224
59242       muslimske    0.322968
13301      undergiven    0.321945
35809      sistnämnde    0.321732
10495       dominanta    0.315187


In [34]:
similars_man_adj_s = n_most_similar(swectors, 10, 'mannen', swectors_filtered)
print(similars_man_adj_s)

filtered
Time elapsed:  2.2308168411254883
                  word  similarity
82054     värnpliktige    0.481998
24762     prostituerad    0.423663
28865         skyldige    0.420797
61483   mordmisstänkte    0.402562
15205             döde    0.400092
3281             andre    0.398971
72226  huvudmisstänkte    0.393731
37183       medåtalade    0.377811
79494      mordåtalade    0.369781
12493      minderåriga    0.359776
4035          sexuellt    0.351787


In [35]:
similars_man_adj_fam = n_most_similar(familjevectors, 10, 'mannen', familje_filtered)
print(similars_man_adj_fam)

filtered
Time elapsed:  1.3085579872131348
             word  similarity
3115        andre    0.426721
10725      äldste    0.383851
2607        lille    0.334098
12527       gamle    0.318669
60637       elake    0.306949
16734      förste    0.302364
2425       äldsta    0.301359
7612   stationära    0.292554
15601      yngste    0.290779
40508      blinde    0.284775
4204       yngsta    0.282013


In [36]:
similars_man_adj_flash = n_most_similar(flashvectors, 10, 'mannen', flash_filtered)
print(similars_man_adj_flash)

filtered
Time elapsed:  2.421638250350952
                 word  similarity
2611            andre    0.474595
108294          nakne    0.416109
35809      sistnämnde    0.409220
18900            fete    0.407243
61824           mörke    0.392290
9628             ende    0.386657
20863        skyldige    0.382801
150112  västerländske    0.380523
25814          starke    0.379380
86966       dominante    0.371991
10478          förste    0.370181
